In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch.autograd import Variable
from math import pi
import random
import numpy as np
from math import cos, sin

# Means of Modeling Manipulators

To start, the classes "Manipulator" and "Link" were created that allows the representation of the specifications of manipulators. These classes will be used later on to generate the data required for the manipulators we will train our models to compute the inverse kinematics for. For an instance of this class to be instantiated, the Denavit-Hartenberg (DH) parameters of the manipulator's links will have to be provided. This allows for the forward kinemtics to be computed in order to find the position of the end effector, given the link's angles.

In [2]:
class Link():
    def __init__(self,theta_offset,d,alpha,a):
        self.theta_offset = theta_offset
        self.d = d
        self.alpha = alpha
        self.a = a
    def get_transformation(self, theta):
        theta = theta+self.theta_offset
        transformation_matrix = np.array([
            [cos(theta), -sin(theta) * cos(self.alpha), sin(theta) * sin(self.alpha), self.a * cos(theta)],
            [sin(theta), cos(theta) * cos(self.alpha), -cos(theta) * sin(self.alpha), self.a * sin(theta)],
            [0, sin(self.alpha), cos(self.alpha), self.d],
            [0, 0, 0, 1]
        ])
        return transformation_matrix

class Manipulator:
    def __init__(self, links=None, q=None):
        self.links = links if links else []
        self.q = q if q else []
    
    def add_link(self, link):
        self.links.append(link)

    def forward_kinematics(self, joint_angles):        
        transformation_matrix = np.identity(4)
        pos = []
        for i in range(len(self.links)):
            transformation_matrix = np.dot(transformation_matrix, self.links[i].get_transformation(joint_angles[i]))
            pos.append([transformation_matrix[0][3],transformation_matrix[1][3]])
        return transformation_matrix

# Two-Joints Manipulator Continous Path

In the second iteration, the aim is to design a neural network capable of determining the **joint angles necessary** for the manipulator to reach a desired end-effector position. This neural network will take as inputs both the desired **end-effector position** as well as the **current joint angles** of the manipulator. The output will be the set of joint angles required to achieve the specified end-effector position.

**Input:**
- eef_x: x component of the end-effector's position (float)
- eef_y: y component of the end-effector's position (float)
- q1_current: current theta_1 angle (float)
- q2_current: current theta_2 angle (float)

**Output:**
- q1_next: next theta_1 angle required to achieve (eef_x,eef_y) position (float)
- q2_next: next theta_2 angle required to achieve (eef_x,eef_y) position (float)

**Initiating a 2-Joints Manipulator**

The class "Manipulator" created was then used to create an instant of a 2 joints model. A sketch of the first manipulator considered can be seen below along with it's DH parameters required to set it up.

<img height="300px"  style="padding:20px" src="Images/two_joints_manip.jpg">
<img height="300px"  style="padding:20px" src="Images/dh_table.jpg">



In [3]:
link1 = Link(theta_offset=0, d=0, alpha=0, a=5)
link2 = Link(theta_offset=0, d=0, alpha=0, a=5)

manip = Manipulator()
manip.add_link(link1)
manip.add_link(link2)

**Data Generation Specifications (exporting data):**

- Range of q1_next: from 0 to 2pi
- Range of q2_next: from 0 to 2pi
- Step Size for both q1_next and q2_next: 0.01

- q1_current and q2_current will be set to 20 random angles +/- a maximum of 0.005 from q1_next and q2_next respectively

(Note: Can skip this block if file "two_joints_with_nearby_data.txt" already in directory)

In [4]:
q1_next_from = 0
q1_next_to = 2*pi
q1_next_step_size = 0.01

q2_next_from = 0
q2_next_to = 2*pi
q2_next_step_size = 0.01

total_iterations = ((q1_next_to - q1_next_from) / q1_next_step_size + 1) * ((q2_next_to - q2_next_from) / q2_next_step_size + 1)
progress_interval = total_iterations // 10 

with open('two_joints_with_nearby_data.txt', 'w') as file:
    q1_next = q1_next_from
    iterations = 0
    while q1_next <= q1_next_to:
        q2_next = q2_next_from
        while q2_next <= q2_next_to:
            end_effector_positions = manip.forward_kinematics([q1_next,q2_next])
            x = end_effector_positions[0][3]
            y = end_effector_positions[1][3]
            for i in range(20):
                ran = random.uniform(-0.005, 0.005)  
                q1_current = q1_next + ran
                ran = random.uniform(-0.005, 0.005)  
                q2_current = q2_next + ran
                file.write(f"{x} {y} {q1_current} {q2_current} {q1_next} {q2_next} \n")
            iterations += 1
            if iterations % progress_interval == 0:
                progress_percent = (iterations / total_iterations) * 100
                print(f"Progress: {progress_percent:.2f}%")
            q2_next = q2_next + q1_next_step_size
        q1_next = q1_next + q2_next_step_size
print(f"Progress: 100.00%")

Progress: 10.00%
Progress: 20.00%
Progress: 30.00%
Progress: 40.00%
Progress: 50.00%
Progress: 60.00%
Progress: 70.00%
Progress: 80.00%
Progress: 90.00%
Progress: 100.00%


**Getting Data (importing data):**

Storing the inputs (eef_x,eef_y,q1_current,q2_current) and outputs (q1_next,q2_next) in the required format

In [49]:
eef_x = []
eef_y = []
q1_current = []
q2_current = []

q1_next = []
q2_next = []

with open('two_joints_with_nearby_data.txt', 'r') as file:
    lines = file.readlines()
    for line in lines:
        data = line.split()
        eef_x.append(float(data[0]))
        eef_y.append(float(data[1]))
        q1_current.append(float(data[2]))
        q2_current.append(float(data[3]))
        q1_next.append(float(data[4]))
        q2_next.append(float(data[5]))

eef_x = np.array(eef_x)
eef_y = np.array(eef_y)
q1_current = np.array(q1_current)
q2_current = np.array(q2_current)

q1_next = np.array(q1_next)
q2_next = np.array(q2_next)

print("eef_x Shape: ", eef_x.shape)
print("eef_y Shape: ", eef_y.shape)
print("q1_current Shape: ", q1_current.shape)
print("q2_current Shape: ", q2_current.shape)

print("q1_next Shape: ", q1_next.shape)
print("q2_next Shape: ", q2_next.shape)

**0. Necessary Setup**

In [6]:
# Using cuda GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
class PosesDataset(Dataset):
    def __init__(self, eef_x, eef_y, q1_current, q2_current, q1_next, q2_next):
        self.eef_x = torch.tensor(eef_x, dtype=torch.float32)
        self.eef_y = torch.tensor(eef_y, dtype=torch.float32)
        self.q1_current = torch.tensor(q1_current, dtype=torch.float32)
        self.q2_current = torch.tensor(q2_current, dtype=torch.float32)
        self.q1_next = torch.tensor(q1_next, dtype=torch.float32)
        self.q2_next = torch.tensor(q2_next, dtype=torch.float32)
        
    def __len__(self):
        return len(self.eef_x)
    
    def __getitem__(self, idx):
        return (
            self.eef_x[idx],
            self.eef_y[idx],
            self.q1_current[idx],
            self.q2_current[idx],
            self.q1_next[idx],
            self.q2_next[idx]
        )

In [46]:
# Initialize dataset
dataset = PosesDataset(eef_x, eef_y, q1_current, q2_current, q1_next, q2_next)

# Define train/validation split (if needed)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Initialize data loaders
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

/var/folders/y_/g46v5wcs2mlg96p8tytg8l7m0000gn/T/ipykernel_34179/3363737747.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.eef_x = torch.tensor(eef_x, dtype=torch.float32)
/var/folders/y_/g46v5wcs2mlg96p8tytg8l7m0000gn/T/ipykernel_34179/3363737747.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.eef_y = torch.tensor(eef_y, dtype=torch.float32)
/var/folders/y_/g46v5wcs2mlg96p8tytg8l7m0000gn/T/ipykernel_34179/3363737747.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.q1_current = torch.tensor(q1_current, dtype=tor

In [11]:
class IKModel(nn.Module):
    def __init__(self):
        super(IKModel, self).__init__()
        # Define your model architecture
        # Example:
        self.fc = nn.Sequential(
            nn.Linear(4, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 2)  # Output layer for q1_next, q2_next
        )
    
    def forward(self, eef_x, eef_y, q1_current, q2_current):
        # Concatenate input features
        input_features = torch.cat((eef_x.unsqueeze(1), eef_y.unsqueeze(1), q1_current.unsqueeze(1), q2_current.unsqueeze(1)), dim=1)
        
        # Forward pass through the model
        output = self.fc(input_features)
        return output


In [43]:
model = IKModel().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
Loss = nn.MSELoss()
print(model)

IKModel(
  (fc): Sequential(
    (0): Linear(in_features=4, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=16, bias=True)
    (7): ReLU()
    (8): Linear(in_features=16, out_features=2, bias=True)
  )
)


In [48]:
num_of_epochs = 1
train_loss_history = []
test_loss_history = []
test_accuracy_history = []

model.train()
for epoch in range(num_of_epochs):
  train_loss = 0.0
  test_loss = 0.0
  for i, batch in enumerate(train_loader):
    eef_x, eef_y, q1_current, q2_current, q1_next, q2_next = batch
    eef_x, eef_y, q1_current, q2_current, q1_next, q2_next = (
            eef_x.to(device).float(),
            eef_y.to(device).float(),
            q1_current.to(device).float(),
            q2_current.to(device).float(),
            q1_next.to(device).float(),
            q2_next.to(device).float(),
        )

    # Forward Pass
    output = model(eef_x, eef_y, q1_current, q2_current)
    # Finding Loss
    combined_q = torch.cat((q1_next.unsqueeze(1), q2_next.unsqueeze(1)), dim=1)
    fit = Loss(output, combined_q)
    print(f"Percentage: {100*i/len(train_loader):.4f}%")
    print("Training Loss: ", fit.item()/len(batch))
    fit.backward()
    # Optimizing Loss
    optimizer.zero_grad()
    optimizer.step()
    train_loss += fit.item()
  model.eval()
  for i, batch in enumerate(test_loader):
    with torch.no_grad():
      eef_x, eef_y, q1_current, q2_current, q1_next, q2_next = batch
      eef_x, eef_y, q1_current, q2_current, q1_next, q2_next = (
              eef_x.to(device).float(),
              eef_y.to(device).float(),
              q1_current.to(device).float(),
              q2_current.to(device).float(),
              q1_next.to(device).float(),
              q2_next.to(device).float(),
          )
      # Forward Pass
      output = model(eef_x, eef_y, q1_current, q2_current)
      # Finding Loss
      combined_q = torch.cat((q1_next.unsqueeze(1), q2_next.unsqueeze(1)), dim=1)
      fit = Loss(output, combined_q)
      # Optimizing Loss
      test_loss += fit.item()
      print(f"Percentage: {100*i/len(test_loader):.4f}%")
      print("Testing Loss: ", fit.item()/len(batch))
  train_loss = train_loss/train_size
  test_loss = test_loss/test_size
  train_loss_history.append(train_loss)
  test_loss_history.append(test_loss)
  print(f"Epoch: {epoch}, Training Loss: {train_loss:.2f}, Training Loss: {train_loss:.4f}")


Percentage: 0.0000%
Training Loss:  1.9175829887390137
Percentage: 0.0000%
Testing Loss:  2.430558204650879
Epoch: 0, Training Loss: 0.41, Training Loss: 0.4109


In [ ]:
fig = plt.figure()
plt.plot(range(30),train_accuracy_history,"-",linewidth=3,label="Train Error")
plt.plot(range(30),test_accuracy_history,"-",linewidth=3,label="Test Error")
plt.legend()
plt.show()

In [ ]:
model.eval()  # Set the model to evaluation mode
all_predicted = []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        output = model(images)
        _, predicted = torch.max(output, 1)
        all_predicted.extend(predicted.cpu().numpy())

# Calculate accuracy using all_predicted and test_set.labels
accuracy = (np.array(all_predicted) == test_set.labels).mean()
print(f"Model Accuracy: {accuracy*100:.2f}%")

In [ ]:
torch.save(model.state_dict(),'drive/MyDrive/Colab-Data/rsp.pt')

In [ ]:
print(model(torch.tensor([0.854494893105,0.98617294483]).to(device)))